## Bedrock 활용 테스트

### 사전 준비사항

- bedrock 콘솔에서 `model access` 신청을 해야 합니다.
- 여기서는 아래 2가지 모델을 주로 활용합니다.
  - `Claude 3 Sonnet` : 한국어를 지원하는 성능이 좋은 LLM입니다.
  - `Titan Text Embeddings V2` : Semantic search 등에 필요한 embedding 모델입니다.
- SageMaker notebook 에서 사용하는 IAM role에 아래 권한을 추가해야 합니다.
  - `AmazonBedrockFullAccess`


In [ ]:
!pip install -q langchain langchain-aws langchain-community

In [ ]:
!pip list | grep 'langchain\|boto3'

In [ ]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime')

### boto3 를 사용하여 bedrock 활용

In [ ]:
bedrock_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

def get_llm_output(prompt):
    body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1024,
                "temperature" : 0.1,
                "top_p": 0.5,
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                        ],
                    }
                ],
            }) 

    response = bedrock.invoke_model(
        body=body, 
        modelId=bedrock_model_id,
        accept='application/json',
        contentType='application/json')

    response_body = json.loads(response.get("body").read())
    llm_output = response_body.get("content")[0].get("text")
    return llm_output

In [ ]:
prompt = "미국에서 가장 큰 도시는 어디인가요?"
llm_out = get_llm_output(prompt)
print(llm_out)

### langchain을 사용하여 bedrock 활용

In [ ]:
from langchain_aws import ChatBedrock

def get_langchain_llm_output(prompt):

    chat = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs={'temperature': 0.1}
    )

    response_text = chat.invoke(prompt).content
    return response_text

In [ ]:
prompt = "미국에서 가장 큰 도시 10개를 알려주세요."
llm_out = get_langchain_llm_output(prompt)
print(llm_out)

### Embedding 모델의 역할

In [ ]:
embedding_model_id = "amazon.titan-embed-text-v2:0"

def get_embedding_output(query):
    
    body = {
        "inputText": query,
        "dimensions": 1024,
        "normalize": True
    }
    
    response = bedrock.invoke_model(
        body=json.dumps(body), 
        modelId=embedding_model_id,
        accept='application/json',
        contentType='application/json')

    response_body = json.loads(response.get("body").read())
    embedding = response_body.get("embedding")
    return embedding

In [ ]:
emb_out = get_embedding_output("토요일에 이마트 영업시간이 어떻게 되나요?")
print(f"Embedding dim: {len(emb_out)}, Sample: {emb_out[0:20]}")

In [ ]:
import numpy as np

def cosine_similarity(list1, list2):
    vec1 = np.array(list1)
    vec2 = np.array(list2)
    
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    cosine_sim = dot_product / (norm_vec1 * norm_vec2)
    return cosine_sim


In [ ]:
sim1 = cosine_similarity(
    get_embedding_output("토요일에 이마트 영업시간이 어떻게되요?"),
    get_embedding_output("주말에 마트 언제까지 문 여나요?"))
sim2 = cosine_similarity(
    get_embedding_output("토요일에 이마트 영업시간이 어떻게되요?"),
    get_embedding_output("토요일에 그 식당에 언제가면 되나요?"))

In [ ]:
print(f"Sim1 : {sim1}, Sim2: {sim2}")